In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

#from btb.tuning import GPTuner, UniformTuner
#from btb import HyperParameter, ParamTypes

# Read Data

In [2]:
folder = 'CMAPSSData'
df_rul = pd.read_table(os.path.join(folder,'RUL_FD001.txt'), header=None)
df_train = pd.read_table(os.path.join(folder, 'train_FD001.txt'), sep=' ', header=None)
df_test = pd.read_table(os.path.join(folder, 'test_FD001.txt'), sep=' ', header=None)

In [3]:
# remove redundant columns due to extra spacing
df_train = df_train[df_train.columns[:26]]
df_test = df_test[df_test.columns[:26]]

In [4]:
# see data structure
print('RUL: ', df_rul.shape)
print('Train: ', df_train.shape)
print('Test: ', df_test.shape)

RUL:  (100, 1)
Train:  (20631, 26)
Test:  (13096, 26)


In [5]:
df_train.head(5)

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


# Feature List

 * 1)	unit number
 * 2)	time, in cycles
 * 3)	operational setting 1
 * 4)	operational setting 2
 * 5)	operational setting 3
 * 6)	sensor measurement  1
 * 7)	sensor measurement  2
 * ...
 * 26)	sensor measurement  26

# Preprocess Train

generate RUL for y-label by reversing the cycle till failure

In [8]:
df_train.rename(columns = {0 : 'unit', 1 : 'cycle'}, inplace = True)

total_cycles = df_train.groupby(['unit']).agg({'cycle' : 'max'}).reset_index()
total_cycles.rename(columns = {'cycle' : 'total_cycles'}, inplace = True)

df_train = df_train.merge(total_cycles, how = 'left', left_on = 'unit', right_on = 'unit')
df_train['RUL'] = df_train.apply(lambda r: int(r['total_cycles'] - r['cycle']), axis = 1)
    

df_train2 = df_train.copy()
del df_train2['cycle']

X_train = df_train2[df_train2.columns[:25]]
y_train = df_train['RUL']

In [9]:
df_train2.head(2)

,unit,2,3,4,5,6,7,8,9,10,...,18,19,20,21,22,23,24,25,total_cycles,RUL
0,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,21.61,...,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,192,191
1,1,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,21.61,...,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,192,190


# Preprocess Test

Get last cycle for each engine to match with the true test_y RUL label

In [10]:
# number of engines
engines = df_test[0].unique()

df_list = []
# get last cycle for each engine
for i in engines:
    df = df_test[df_test[0]==i]
    last = (df[-1:])
    df_list.append(last)

# union all rows in a dataframe
X_test = pd.concat(df_list)
del X_test[1]

In [11]:
y_test = df_rul.values.flatten()

In [12]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

# Fit Model

In [13]:
model = RandomForestRegressor(n_estimators=1056, max_depth=9, n_jobs=-1)
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=9,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1056,
                      n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

## SVM

In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

model = SVR(gamma='auto')
parameters = {'kernel':('rbf'), 'C':[0.8, 1, 1.2, 1.5, 2]}
clf = GridSearchCV(model, parameters, verbose=2)
clf.fit(X_train, y_train)

y_predicted_train = clf.predict(X_train)
y_predicted_test = clf.predict(X_test)

/Users/vrishank/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=None)]: Done  36 out of  36 | elapsed:  8.4min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto', kernel='rbf', max_iter=-1,
                           shrinking=True, tol=0.001, verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.1, 1, 5, 10],
                         'kernel': ('linear', 'rbf', 'poly')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=1)

In [24]:
y_predicted_train = clf.predict(X_train)
y_predicted_test = clf.predict(X_test)

MAE uses the absolute error, ie. # of cycles

In [25]:
MAE_train = mean_absolute_error(y_train, y_predicted_train)
MAE_test = mean_absolute_error(y_test, y_predicted_test)

print('Train MAE', MAE_train)
print('Train MAE', MAE_test)

Train MAE 29.27286832915188
Train MAE 21.13640503413754


In [26]:
MSE_train = mean_squared_error(y_train, y_predicted_train)
MSE_test = mean_squared_error(y_test, y_predicted_test)

print('Train RMSE:', np.sqrt(MSE_train))
print('Test RMSE:', np.sqrt(MSE_test))

Train RMSE: 41.941195000786884
Test RMSE: 26.342764337340384


## Random Forest

In [30]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs=-1)
parameters = {'max_depth':[5, 7, 9, 11] , 'n_estimators':[50, 100, 250, 500, 1000]}
clf = GridSearchCV(model, parameters, verbose=2)
clf.fit(X_train, y_train)

y_predicted_train = clf.predict(X_train)
y_predicted_test = clf.predict(X_test)

MAE_train = mean_absolute_error(y_train, y_predicted_train)
MAE_test = mean_absolute_error(y_test, y_predicted_test)

print('Train MAE', MAE_train)
print('Train MAE', MAE_test)

MSE_train = mean_squared_error(y_train, y_predicted_train)
MSE_test = mean_squared_error(y_test, y_predicted_test)

print('Train RMSE:', np.sqrt(MSE_train))
print('Test RMSE:', np.sqrt(MSE_test))

/Users/vrishank/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] max_depth=5, n_estimators=50 ....................................
[CV] ..................... max_depth=5, n_estimators=50, total=   1.7s
[CV] max_depth=5, n_estimators=50 ....................................


[Parallel(n_jobs=None)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s


[CV] ..................... max_depth=5, n_estimators=50, total=   1.8s
[CV] max_depth=5, n_estimators=50 ....................................
[CV] ..................... max_depth=5, n_estimators=50, total=   1.4s
[CV] max_depth=5, n_estimators=100 ...................................
[CV] .................... max_depth=5, n_estimators=100, total=   3.2s
[CV] max_depth=5, n_estimators=100 ...................................
[CV] .................... max_depth=5, n_estimators=100, total=   2.8s
[CV] max_depth=5, n_estimators=100 ...................................
[CV] .................... max_depth=5, n_estimators=100, total=   2.8s
[CV] max_depth=5, n_estimators=250 ...................................
[CV] .................... max_depth=5, n_estimators=250, total=   6.3s
[CV] max_depth=5, n_estimators=250 ...................................
[CV] .................... max_depth=5, n_estimators=250, total=   7.0s
[CV] max_depth=5, n_estimators=250 ...................................
[CV] .

[CV] .................. max_depth=11, n_estimators=1000, total=  53.6s


[Parallel(n_jobs=None)]: Done  60 out of  60 | elapsed: 16.9min finished


Train MAE 30.38278829371944
Train MAE 29.96958981052097
Train RMSE: 41.75586348547492
Test RMSE: 39.72114621380167


## GBT

In [30]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor()
parameters = {'max_depth':[5, 7, 9, 11] , 'n_estimators':[50, 100, 250, 500, 1000]}
clf = GridSearchCV(model, parameters, verbose=2, n_jobs=-1)
clf.fit(X_train, y_train)

y_predicted_train = clf.predict(X_train)
y_predicted_test = clf.predict(X_test)

MAE_train = mean_absolute_error(y_train, y_predicted_train)
MAE_test = mean_absolute_error(y_test, y_predicted_test)

print('Train MAE', MAE_train)
print('Train MAE', MAE_test)

MSE_train = mean_squared_error(y_train, y_predicted_train)
MSE_test = mean_squared_error(y_test, y_predicted_test)

print('Train RMSE:', np.sqrt(MSE_train))
print('Test RMSE:', np.sqrt(MSE_test))

Train MAE 48.87750701893625
Train MAE 34.918492508487
Train RMSE: 61.107487716514584
Test RMSE: 43.83121657104109


## KNN

In [33]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

model = KNeighborsRegressor(n_jobs=-1)
parameters = {'n_neighbors':[2, 4, 8, 16, 32, 64, 128]}
clf = GridSearchCV(model, parameters, verbose=2)
clf.fit(X_train, y_train)

y_predicted_train = clf.predict(X_train)
y_predicted_test = clf.predict(X_test)

MAE_train = mean_absolute_error(y_train, y_predicted_train)
MAE_test = mean_absolute_error(y_test, y_predicted_test)

print('Train MAE', MAE_train)
print('Train MAE', MAE_test)

MSE_train = mean_squared_error(y_train, y_predicted_train)
MSE_test = mean_squared_error(y_test, y_predicted_test)

print('Train RMSE:', np.sqrt(MSE_train))
print('Test RMSE:', np.sqrt(MSE_test))

/Users/vrishank/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


[CV] n_neighbors=2 ...................................................
[CV] n_neighbors=2 ...................................................
[CV] n_neighbors=2 ...................................................
[CV] n_neighbors=4 ...................................................
Fitting 3 folds for each of 7 candidates, totalling 21 fits


/Users/vrishank/anaconda3/lib/python3.6/site-packages/sklearn/neighbors/base.py:420: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)
/Users/vrishank/anaconda3/lib/python3.6/site-packages/sklearn/neighbors/base.py:420: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)
/Users/vrishank/anaconda3/lib/python3.6/site-packages/sklearn/neighbors/base.py:420: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)
/Users/vrishank/anaconda3/lib/python3.6/site-packages/sklearn/neighbors/base.py:420: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)


[CV] .................................... n_neighbors=2, total=  12.2s
[CV] n_neighbors=4 ...................................................


/Users/vrishank/anaconda3/lib/python3.6/site-packages/sklearn/neighbors/base.py:420: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)


...

[CV] ................................... n_neighbors=64, total=  23.5s
[CV] n_neighbors=128 .................................................


/Users/vrishank/anaconda3/lib/python3.6/site-packages/sklearn/neighbors/base.py:420: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  n_jobs = effective_n_jobs(self.n_jobs)


[CV] ................................... n_neighbors=64, total=  24.8s
[CV] .................................. n_neighbors=128, total=  25.0s
[CV] .................................. n_neighbors=128, total=  23.4s
[CV] .................................. n_neighbors=128, total=  10.4s


[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  1.6min finished


Train MAE 29.70165610913189
Train MAE 31.37953125
Train RMSE: 40.90244824767529
Test RMSE: 38.52132380507337


## MLP

In [ ]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

from sklearn.neural_network import MLPRegressor

model = MLPRegressor(hidden_layer_sizes=(150,100,50), max_iter=100,activation = 'relu',solver='adam',random_state=1)
model.fit(X_train, y_train)

y_predicted_train = model.predict(X_train)
y_predicted_test = model.predict(X_test)

MAE_train = mean_absolute_error(y_train, y_predicted_train)
MAE_test = mean_absolute_error(y_test, y_predicted_test)

print('Train MAE', MAE_train)
print('Train MAE', MAE_test)

MSE_train = mean_squared_error(y_train, y_predicted_train)
MSE_test = mean_squared_error(y_test, y_predicted_test)

print('Train RMSE:', np.sqrt(MSE_train))
print('Test RMSE:', np.sqrt(MSE_test))

Train MAE 24.278582307600132
Train MAE 25.398724428038605
Train RMSE: 34.3964518896602
Test RMSE: 37.305551637243134


/Users/vrishank/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [10]:
from sklearn.neighbors import KNeighborsRegressor


model = KNeighborsRegressor(n_neighbors=8)
model.fit(X_train, y_train)

y_predicted_train = model.predict(X_train)
y_predicted_test = model.predict(X_test)

MAE_train = mean_absolute_error(y_train, y_predicted_train)
MAE_test = mean_absolute_error(y_test, y_predicted_test)

print('Train MAE', MAE_train)
print('Train MAE', MAE_test)

MSE_train = mean_squared_error(y_train, y_predicted_train)
MSE_test = mean_squared_error(y_test, y_predicted_test)

print('Train RMSE:', np.sqrt(MSE_train))
print('Test RMSE:', np.sqrt(MSE_test))

Train MAE 24.34629319955407
Train MAE 29.645
Train RMSE: 33.98290621145233
Test RMSE: 41.02928070293214
